# Setting Up the Environment

In [3]:
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# install nnunetv2
!pip install nnunetv2

  Using cached nnunetv2-2.6.2.tar.gz (211 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached acvl_utils-0.2.5.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Using cached dynamic_network_architectures-0.4.2.tar.gz (28 kB)
  Preparing metadata (setup.py) ... done
  Using cached batchgenerators-0.25.1.tar.gz (76 kB)
  Preparing metadata (setup.py) ... done
  Using cached imagecodecs-2025.8.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (20 kB)
  Using cached yacs-0.1.8-py3-none-any.whl.metadata (639 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.

In [4]:
# check nnunetv2
import nnunetv2

In [5]:
# Make Directory to store Models
!mkdir -p /kaggle/working/nnUNet_models/Dataset500_KiTS2023

In [6]:
# Make Directory to align with nnUNet
!mkdir -p /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/

In [7]:
# Make Directory to align with nnUNet
!mkdir -p /kaggle/working/nnUNet_preprocessed

In [8]:
import os

# Raw data (can be empty if you only do inference)
os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw"

# Preprocessed data (can be empty if you only do inference)
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"

# Model results folder (where your Dataset700_MYTASK lives)
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_models"


In [ ]:
# #Code to remove file
# import os

# file_path = "/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imaging_001_0000.nii.gz"

# if os.path.exists(file_path):
#     os.remove(file_path)
#     print("File deleted:", file_path)
# else:
#     print("File not found:", file_path)


In [11]:
# # Code to remove folder
# import shutil

# folder_path = "/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr"

# # Remove the folder and everything inside
# shutil.rmtree(folder_path)


# Data Loading & Pre-Processing

In [ ]:
import os
import SimpleITK as sitk

# === Directories ===
input_base = '/kaggle/input/nnunet-training/first50/imagesTr'
output_base = '/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTr'
os.makedirs(output_base, exist_ok=True)

# === Range of cases to convert ===
for i in range(0, 40):  
    case_id = f"imaging_{i:03d}_0000.nii"
    input_folder = os.path.join(input_base, case_id)
    input_file = os.path.join(input_folder, "imaging.nii")
    output_file = os.path.join(output_base, f"{case_id}.gz")  # final output: imaging_000_0000.nii.gz
    
    print(f"\n Processing: {input_file}")
    
    try:
        # Read input .nii
        img = sitk.ReadImage(input_file)
        
        # Write compressed .nii.gz
        writer = sitk.ImageFileWriter()
        writer.SetFileName(output_file)
        writer.UseCompressionOn()
        writer.Execute(img)
        
        # Show success message
        print(f" Saved compressed file: {output_file}")
        
        # Optional: check size
        original_size = os.path.getsize(input_file) / (1024 * 1024)
        compressed_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"   Original: {original_size:.2f} MB | Compressed: {compressed_size:.2f} MB")
        
    except Exception as e:
        print(f" Error processing {input_file}: {e}")

In [ ]:
import os
import SimpleITK as sitk

# === Directories ===
input_base = '/kaggle/input/nnunet-training/first50/labelsTr'
output_base = '/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/labelsTr'
os.makedirs(output_base, exist_ok=True)

# === Range of cases to convert ===
for i in range(0, 40):  
    case_id = f"imaging_{i:03d}.nii"
    input_folder = os.path.join(input_base, f"imaging_{i:03d}.nii")
    input_file = os.path.join(input_folder, "segmentation.nii")
    output_file = os.path.join(output_base, f"{case_id}.gz")  # final output: imaging_000_0000.nii.gz
    
    print(f"\n Processing: {input_file}")
    
    try:
        # Read input .nii
        img = sitk.ReadImage(input_file)
        
        # Write compressed .nii.gz
        writer = sitk.ImageFileWriter()
        writer.SetFileName(output_file)
        writer.UseCompressionOn()
        writer.Execute(img)
        
        # Show success message
        print(f" Saved compressed file: {output_file}")
        
        # Optional: check size
        original_size = os.path.getsize(input_file) / (1024 * 1024)
        compressed_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"   Original: {original_size:.2f} MB | Compressed: {compressed_size:.2f} MB")
        
    except Exception as e:
        print(f" Error processing {input_file}: {e}")

In [ ]:
!cp /kaggle/input/nnunet-training/first50/dataset.json /kaggle/working/nnUNet_raw/Dataset500_KiTS2023

In [ ]:
# Pre processing with reduced workload
import os

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["nnUNet_tmp"] = "/content/tmp"
os.makedirs("/content/tmp", exist_ok=True)

In [ ]:
# Data Pre Processing with reduce Workload
!nnUNetv2_plan_and_preprocess -d 500 -c 3d_fullres --verify_dataset_integrity --verbose -np 2 -pl nnUNetPlannerResEncM

# Model Training

In [ ]:
!nnUNetv2_train Dataset500_KiTS2023 3d_fullres all -p nnUNetResEncUNetMPlans -tr nnUNetTrainer_10epochs

In [ ]:
# import nibabel as nib
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.widgets import Slider
# from matplotlib.colors import ListedColormap
# from matplotlib.patches import Patch


# # Paths
# ct_path = "/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imaging_001_0000.nii.gz"
# seg_path = "/kaggle/working/results/imaging_001.nii.gz"

# # Load images
# ct_img = nib.load(ct_path)
# ct_data = ct_img.get_fdata()

# seg_img = nib.load(seg_path)
# seg_data = seg_img.get_fdata().astype(int)

# # --- 1. Custom Colors and Masking (Same as before) ---
# colors = ['#000000', '#9467BD', '#00BCD4', '#8BC34A']
# cmap = ListedColormap(colors)
# masked_seg_data = np.ma.masked_where(seg_data == 0, seg_data)
# cmap.set_bad(color='none', alpha=0.0)

# # --- 2. Function to Get Correct Axial Slice ---
# def get_axial_slice(data_3d, index, axis=0):
#     if axis == 0:
#         slice_2d = data_3d[index, :, :]
#     elif axis == 1:
#         slice_2d = data_3d[:, index, :]
#     else:
#         slice_2d = data_3d[:, :, index]
    
#     # FIX ROTATION: Use np.rot90 instead of .T to correct orientation
#     # k=1 rotates 90 degrees CCW, k=3 rotates 270 degrees CCW (90 degrees CW)
#     # This value is often found through trial and error (try k=1, 2, or 3)
#     return np.rot90(slice_2d, k=1) 

# def get_masked_axial_slice(data_3d, index, axis=0):
#     if axis == 0:
#         masked_slice = masked_seg_data[index, :, :]
#     elif axis == 1:
#         masked_slice = masked_seg_data[:, index, :]
#     else:
#         masked_slice = masked_seg_data[:, :, index]
        
#     # FIX ROTATION: Apply the same rotation
#     return np.rot90(masked_slice, k=1) 

# # --- 3. Initial Axial Slice Setup ---
# AXIAL_AXIS = 0 
# MAX_SLICES = ct_data.shape[AXIAL_AXIS]
# # FIX COLORS: Set initial slice to a value where structures are present (e.g., 150)
# slice_idx = 260

# # Plot setup
# fig, ax = plt.subplots(figsize=(6,6))
# plt.subplots_adjust(bottom=0.25)

# # Plot the corrected axial slice data
# img_plot = ax.imshow(get_axial_slice(ct_data, slice_idx, axis=AXIAL_AXIS), cmap='gray', vmin=-100, vmax=300)
# overlay = ax.imshow(get_masked_axial_slice(seg_data, slice_idx, axis=AXIAL_AXIS), cmap=cmap, alpha=0.5, interpolation='none')

# ax.set_title(f'Axial Slice {slice_idx} of {MAX_SLICES-1}')
# ax.axis('off')

# # --- 4. Slider and Update Function (Remains the same) ---
# ax_slider = plt.axes([0.25, 0.1, 0.5, 0.03])
# slider = Slider(ax_slider, 'Axial Slice', 0, MAX_SLICES-1, valinit=slice_idx, valstep=1)

# def update(val):
#     idx = int(slider.val)
#     img_plot.set_data(get_axial_slice(ct_data, idx, axis=AXIAL_AXIS))
#     overlay.set_data(get_masked_axial_slice(seg_data, idx, axis=AXIAL_AXIS))
#     ax.set_title(f'Axial Slice {idx} of {MAX_SLICES-1}')
#     fig.canvas.draw_idle()

# slider.on_changed(update)

# # --- 5. Legend (Same as before) ---
# legend_elements = [
#     Patch(facecolor='#9467BD', alpha=0.5, label='Kidney (1)'),
#     Patch(facecolor='#00BCD4', alpha=0.5, label='Tumor (2)'),
#     Patch(facecolor='#8BC34A', alpha=0.5, label='Cyst (3)')
# ]
# ax.legend(handles=legend_elements, loc='lower right', bbox_to_anchor=(1.0, 0.0), framealpha=0.6)

# plt.show()

# Prediction & Scores

In [15]:
# Loading the Test Images
import os
import SimpleITK as sitk

# === Directories ===
input_base = '/kaggle/input/nnunet-training/first50/imagesTs'
output_base = '/kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs'
os.makedirs(output_base, exist_ok=True)

# === Range of cases to convert ===
for i in range(40, 50):  
    case_id = f"imaging_{i:03d}_0000.nii"
    input_folder = os.path.join(input_base, case_id)
    input_file = os.path.join(input_folder, "imaging.nii")
    output_file = os.path.join(output_base, f"{case_id}.gz")  # final output: imaging_000_0000.nii.gz
    
    print(f"\n Processing: {input_file}")
    
    try:
        # Read input .nii
        img = sitk.ReadImage(input_file)
        
        # Write compressed .nii.gz
        writer = sitk.ImageFileWriter()
        writer.SetFileName(output_file)
        writer.UseCompressionOn()
        writer.Execute(img)
        
        # Show success message
        print(f" Saved compressed file: {output_file}")
        
        # Optional: check size
        original_size = os.path.getsize(input_file) / (1024 * 1024)
        compressed_size = os.path.getsize(output_file) / (1024 * 1024)
        print(f"   Original: {original_size:.2f} MB | Compressed: {compressed_size:.2f} MB")
        
    except Exception as e:
        print(f" Error processing {input_file}: {e}")


 Processing: /kaggle/input/nnunet-training/first50/imagesTs/imaging_040_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs/imaging_040_0000.nii.gz
   Original: 414.00 MB | Compressed: 61.04 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTs/imaging_041_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs/imaging_041_0000.nii.gz
   Original: 104.00 MB | Compressed: 19.07 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTs/imaging_042_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs/imaging_042_0000.nii.gz
   Original: 602.00 MB | Compressed: 98.67 MB

 Processing: /kaggle/input/nnunet-training/first50/imagesTs/imaging_043_0000.nii/imaging.nii
 Saved compressed file: /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs/imaging_043_0000.nii.gz
   Original: 344.00 MB | Compressed: 52.88 MB

 Processing: /kaggle/in

In [17]:
!cp -r /kaggle/input/nnunet2nd/pytorch/default/1/pretrained_models/nnUNetTrainer__nnUNetPlans__3d_fullres_batch_4_all /kaggle/working/nnUNet_models/Dataset500_KiTS2023

In [19]:
!cp -r /kaggle/input/nnunet2nd/pytorch/default/1/pretrained_models/nnUNetTrainer__nnUNetPlans__3d_lowres_plain_all /kaggle/working/nnUNet_models/Dataset500_KiTS2023

In [20]:
!cp -r /kaggle/input/nnunet2nd/pytorch/default/1/pretrained_models/nnUNetTrainer__nnUNetPlans__3d_lowres_residual_all /kaggle/working/nnUNet_models/Dataset500_KiTS2023

In [21]:
# Dir. to store inferencing results
!mkdir -p /kaggle/working/inference_outs

In [31]:
# Run the Prediction 
!nnUNetv2_predict -i /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs -o /kaggle/working/inference_outs/Lowres_outs -d 500 -c 3d_lowres_plain_all -f all -chk checkpoint_best.pth


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
There are 10 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we star

In [28]:
# Run the Prediction 
!nnUNetv2_predict -i /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs -o /kaggle/working/inference_outs/Lowres_residual_outs -d 500 -c 3d_lowres_residual_all -f all


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/get_network_from_plans.py:20: UserWar

In [24]:
# Run the Prediction 
!nnUNetv2_predict -i /kaggle/working/nnUNet_raw/Dataset500_KiTS2023/imagesTs -o /kaggle/working/inference_outs/Fullres_outs -d 500 -c 3d_fullres_batch_4_all -f all


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

/usr/local/lib/python3.11/dist-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
There are 10 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we star

In [ ]:

# kits23_compute_metrics FOLDER_WITH_PREDICTIONS -num_processes XX